In [1]:
from app.generate_diff import find_relevant_files_for_prompt_and_repo

processedFiles = await find_relevant_files_for_prompt_and_repo("The file where the routes are defined. Should contain things like app.get", "https://github.com/chasemc67/TinyGen")
print(processedFiles)

[ProcessedGithubFile(originalFile=GithubFile(filename='.gitignore', content='# Config secrets\n.env\n\n# Python cache files\n__pycache__/\n*.py[cod]\n\n# Virtual environment\nvenv/\n\n# Project-specific settings\n.env  # environment variables, never commit this\n*.log  # log files\n\n# Byte-compiled / optimized / DLL files\n*.py[cod]\n*$py.class\n\n# Distribution / packaging\n.Python\nbuild/\ndevelop-eggs/\ndist/\ndownloads/\neggs/\n.eggs/\nlib/\nlib64/\nparts/\nsdist/\nvar/\n*.egg-info/\n.installed.cfg\n*.egg\n\n# PyCharm files\n.idea/\n*.iml'), llmResponse='\n    0'), ProcessedGithubFile(originalFile=GithubFile(filename='Procfile', content='web: uvicorn main:app --host=0.0.0.0 --port=${PORT:-5000}'), llmResponse='\n    0'), ProcessedGithubFile(originalFile=GithubFile(filename='README.md', content='# TinyGen\n\nA simple API to generate code diffs using ChatGPT\n\n## Running the Application Locally\n\n1. First, clone the repository to your local machine and navigate to the project dire

In [2]:
for file in processedFiles:
    print(file.originalFile.filename, file.llmResponse)
    print("\n")

.gitignore 
    0


Procfile 
    0


README.md 
    0


config.py 
    0


main.py 
    50


requirements.txt 
    0


runtime.txt  0


testbed.ipynb 
    0


